# Partie 3 : Recommandation (12 points)

### 1. Qu'est-ce qui est recommandé ?
Nous recommandons des streamers que l'utilisateur pourrait apprécier en fonction des habitudes de visionnage des autres utilisateurs ayant des préférences similaires.

### 2. À qui faites-vous cette recommandation ?
Nous voulons faire ces recommandations à un utilisateur spécifique.

### 3. Approche et logique
Nous utilisons une approche basée sur la similarité collaborative :
 
Si deux utilisateurs regardent les mêmes streamers, ils partagent des intérêts communs. Nous recommandons à un utilisateur des streamers populaires chez d'autres utilisateurs ayant des préférences similaires.

Étapes :

- Nous devons identifier les utilisateurs ayant regardé des streamers communs avec l'utilisateur cible.
- Ensuite, trouver les streamers regardés par ces utilisateurs similaires mais pas encore regardés par l'utilisateur cible.
- Calculer un score pour chaque streamer basé sur leur popularité chez les utilisateurs similaires.
- Pour finir, nous allons retourner les streamers les mieux classés comme recommandations.

### 4. La requête Cyhper

MATCH (user:User {user_id: 721})-[:WATCHED]->(stream:Stream)<-[:WATCHED]-(otherUser:User)
WHERE user <> otherUser
WITH user, otherUser, COUNT(stream) AS sharedStreams
ORDER BY sharedStreams DESC
LIMIT 10

MATCH (otherUser)-[:WATCHED]->(otherStream:Stream)-[:HOSTED_BY]->(streamer:Streamer)
WHERE NOT EXISTS {
    MATCH (user)-[:WATCHED]->(otherStream)
}
WITH streamer, COUNT(DISTINCT otherUser) AS popularity
RETURN streamer.username AS RecommendedStreamer, popularity
ORDER BY popularity DESC
LIMIT 5

### 5. Explications

#### a. Trouver les utlisateurs similaires

MATCH (user:User {user_id: 721})-[:WATCHED]->(stream:Stream)<-[:WATCHED]-(otherUser:User)
WHERE user <> otherUser
WITH otherUser, COUNT(stream) AS sharedStreams
ORDER BY sharedStreams DESC
LIMIT 10

* On identifie les utilisateurs (otherUser) qui ont regardé les mêmes streams que l'utilisateur cible.
* On compte le nombre de streams partagés (sharedStreams) et sélectionne les 10 utilisateurs les plus similaires.

#### b. Identifier les streams non visionnés

MATCH (otherUser)-[:WATCHED]->(otherStream:Stream)-[:HOSTED_BY]->(streamer:Streamer)
WHERE NOT EXISTS {
    MATCH (user)-[:WATCHED]->(otherStream)
}

* On explore les streams regardés par les utilisateurs similaires (otherUser) qui n'ont pas encore été regardés par l'utilisateur cible.

#### c. Calculer la popularité des streamers 

WITH streamer, COUNT(otherUser) AS popularity
RETURN streamer.username AS RecommendedStreamer, popularity
ORDER BY popularity DESC
LIMIT 5

* On compte combien d'utilisateurs similaires ont regardé chaque streamer pour mesurer leur popularité.
* On retourne les 5 streamers les plus populaires comme recommandations.
